In [1]:
import vtk
import numpy as np
import SimpleITK as sitk
import matplotlib.pyplot as plt

#### Using the segmentation mask registered to the individual images, render and compute the volume certain anatomical features
1. Segment the structure of interest - simple since you know the pxl value
2. Compute a marching cubes surface on the segmented region
3. Render the volume/surface visualization
4. Compute the volume

In [18]:
# Create a dict of labels and their value
custom_labels = ['fourth ventricle', 'amygdala', 'third ventricle', 'hippocampus', 'basal forebrain', 'thalamus', 'lateral ventricle', 'optic tract', 'arbor vita of cerebellum', 'nucleus accumbens', 'globus pallidus']
custom_labels_LR = [side_label for label in custom_labels for side_label in [label + " R", label + " L"]]
common_stuct_start_idx = 251
common_struct = {}

for i, label in enumerate(custom_labels_LR):
    common_struct[label] = common_stuct_start_idx + i

print(common_struct)

# VTK helpers
# Returns the window, renderer, and interactor
def create_render_objs():
    window = vtk.vtkRenderWindow()

    interactor = vtk.vtkRenderWindowInteractor()
    renderer = vtk.vtkRenderer()

    window.AddRenderer(renderer)
    interactor.SetRenderWindow(window)

    return window, renderer, interactor


{'fourth ventricle R': 251, 'fourth ventricle L': 252, 'amygdala R': 253, 'amygdala L': 254, 'third ventricle R': 255, 'third ventricle L': 256, 'hippocampus R': 257, 'hippocampus L': 258, 'basal forebrain R': 259, 'basal forebrain L': 260, 'thalamus R': 261, 'thalamus L': 262, 'lateral ventricle R': 263, 'lateral ventricle L': 264, 'optic tract R': 265, 'optic tract L': 266, 'arbor vita of cerebellum R': 267, 'arbor vita of cerebellum L': 268, 'nucleus accumbens R': 269, 'nucleus accumbens L': 270, 'globus pallidus R': 271, 'globus pallidus L': 272}


In [ ]:
# select structure to render the volume of
struct_idx = 0 #(list(common_struct)).index(visualize_stuct)
structs = list(common_struct.keys())
visualize_stuct = structs[struct_idx] #'hippocampus R'
thresh_value = 48#common_struct[visualize_stuct]

print("struct idx:", struct_idx)

#human_labels_image_path = "./atlases/human_mni_icbm152_CerebrA_tal_nlin_sym_09c.nii" 
human_labels_image_path = "./atlases/human_atlas_resampled_bet.nii" 
mouse_labels_image_path = "./atlases/mouse_labels_common_proc.nii"

image_path = human_labels_image_path

reader_type = "NIFTY"

# Set morphological operation properties
dilate_value = 1
erode_value = 0
kernel_size = (3, 3, 3)

############################################## params done

# read image
if reader_type == "DICOM":
    dicom_reader = vtk.vtkDICOMImageReader()
    dicom_reader.SetDirectoryName(image_path)
    reader = dicom_reader
else:
    nifti_reader = vtk.vtkNIFTIImageReader()
    nifti_reader.SetFileName(image_path)
    reader = nifti_reader

reader.Update()

thresh = vtk.vtkImageThreshold()
thresh.SetInputConnection(reader.GetOutputPort())
thresh.ThresholdBetween(thresh_value, thresh_value)
thresh.ReplaceInOn()
thresh.SetInValue(1)  # magic number rule exception: will never need to be modified
thresh.ReplaceOutOn()
thresh.SetOutValue(0)  # magic number rule exception: will never need to be modified
thresh.SetOutputScalarTypeToFloat()
thresh.Update()

# Dilate the image

dilate = vtk.vtkImageDilateErode3D()
dilate.SetInputConnection(thresh.GetOutputPort())
dilate.SetKernelSize(*kernel_size)
dilate.SetDilateValue(dilate_value)
dilate.SetErodeValue(erode_value)
dilate.Update()

# convert the thresholded anatomical structure top a surface object
isosurface = vtk.vtkMarchingCubes()
isosurface.SetInputConnection(dilate.GetOutputPort())
isosurface.SetValue(0, 1) # 1 since that's what the thresholding replaced???
isosurface.Update()

mapper = vtk.vtkPolyDataMapper()
mapper.SetInputConnection(isosurface.GetOutputPort())
mapper.ScalarVisibilityOff()

# set colour to match labels rendering
color_series = vtk.vtkColorSeries()
# choose the vtk colour scheme
color_series.SetColorScheme(vtk.vtkColorSeries.BREWER_QUALITATIVE_SET3)

# skip over grey (idx = 8) in colour series
grey_offset = 0
num_colours_in_map = color_series.GetNumberOfColors()

if struct_idx >= 8:
    grey_offset = 1
if struct_idx >= 8 + num_colours_in_map - 1:
    grey_offset = 2
print(grey_offset)

colour = color_series.GetColor((struct_idx + grey_offset)  % num_colours_in_map)

# for first loop over colours
if struct_idx < (num_colours_in_map - 1):
    scale = 255
elif struct_idx < 2*(num_colours_in_map - 1):
    scale = 150
else:
    scale = 5

colour = (colour.GetRed() / scale, colour.GetGreen() / scale, colour.GetBlue() / scale)

# Create a vtkMassProperties object to calculate the volume
mass_properties = vtk.vtkMassProperties()
mass_properties.SetInputData(isosurface.GetOutput())

# Compute the volume
volume = mass_properties.GetVolume()

text_actor = vtk.vtkTextActor()
text_actor.GetTextProperty().SetFontSize(24)
text_actor.GetTextProperty().SetColor(1, 1, 1)
text_actor.SetInput(str(visualize_stuct) + " (Volume: " + str(round(volume, 3)) + ")")
text_property = text_actor.GetTextProperty()
text_property.SetFontSize(16)

print(f"Volume: {volume}")

actor = vtk.vtkActor()
actor.SetMapper(mapper)
actor.GetProperty().SetColor(*colour)

# Standard rendering set-up objects
window, renderer, interactor = create_render_objs()

# Add actor to the renderer
renderer.AddActor(actor)
renderer.AddActor(text_actor)
    
# Display
window.Render()
interactor.Initialize()
interactor.Start()

